In [1]:
#计算finger efficiency：key之间距离/path长度 (histogram)
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
def readPickle(filename):
    import pickle

    # reload a file to a variable
    with open(filename, 'rb') as file:
        a_dict1 =pickle.load(file)

    return a_dict1

In [3]:
def writePickle(filename,dictionary):
    import pickle
    # pickle a variable to a file
    file = open(filename, 'wb')
    pickle.dump(dictionary, file)
    file.close()

In [4]:
def KeyToXY(key):
    keys_ = ['q','w','e','r','t','y','u','i','o','p','å','a','s','d','f','g','h','j','k','l','ö','ä','z'
            ,'x','c','v','b','n','m','<',' ','enter']
    xs = [566, 697, 828, 959, 1090, 1221, 1352, 1483, 1614, 1745, 1876, 566, 697, 828, 959
         , 1090, 1221, 1352, 1483, 1614, 1745, 1876, 828, 959, 1090, 1221, 1352, 1483, 1614
         , 1810, 1193, 1737]
    ys = [1955, 1955, 1955, 1955, 1955, 1955, 1955, 1955, 1955, 1955, 1955, 2185, 2185
         , 2185, 2185, 2185, 2185, 2185, 2185, 2185, 2185, 2185, 2415, 2415, 2415, 2415
         , 2415, 2415, 2415, 2415, 2645, 2645]

    if key not in keys_:
        return []
    else:
        p = keys_.index(key)
        return [xs[p], ys[p]]

In [5]:
def dis(posX1, posY1, posX2, posY2):
    import math
    
    ofx = posX2-posX1
    ofy = posY2-posY1
    return [math.sqrt(ofx*ofx+ofy*ofy),ofx,ofy]

In [6]:
#dictFix = readPickle('./pkl/eyeMovementData.pkl')
dictFin = readPickle('./pkl/fingerMovementData.pkl')

In [7]:
def drawGraph(xlist, ylist, title, xlable,ylable):
    import matplotlib.pyplot as plt
  
    t = title
    
    #去掉 IKI 结果中的 outlier
    #temp = rejectOutliers(xlist, ylist)
    #xlist1 = temp[0]
    #ylist1 = temp[1]
    
    
    yfit = []
    #a,b = np.polyfit(xlist, ylist, 1)
    slope, intercept, r_value, p_value, std_err = stats.linregress(xlist, ylist)
    print('Corr = '+str(r_value))
    
    for item in xlist:
        yfit.append(slope*item+intercept)
   
    plt.xlabel(xlable)  
    plt.ylabel(ylable)
    plt.title(t)
    
    
    plt.plot(xlist, ylist ,'ro', alpha=.5,label = 'Corr:'+str(round(r_value,2)))
    #plt.plot(xlist1, ylist1 ,'ro', alpha=.5)
    
    plt.plot(xlist,yfit, linewidth = 2)


    plt.savefig(t+".png")
   
    plt.show()

In [8]:
def rejectOutliers1(dataY):
    u = np.mean(dataY)
    s = np.std(dataY)
    dataY1 = []
    rejected = []
    for i in range(len(dataY)):
        if (u - 2 * s) < dataY[i] < (u + 2 * s):
            dataY1.append(dataY[i])
        else:
            rejected.append(dataY[i])   
    if len(dataY)!=len(dataY1):
        print('outlier rejected:')
        print(rejected)
    return dataY1

In [9]:
def rejectOutliers2(dataX, dataY):
    u = np.mean(dataY)
    s = np.std(dataY)
    print([u,s])
    dataX1 = []
    dataY1 = []
    rejected = []
    for i in range(len(dataY)):
        if (u - 2 * s) < dataY[i] < (u + 2 * s):
            dataX1.append(dataX[i])
            dataY1.append(dataY[i])
        else:
            rejected.append(dataY[i])   
    if len(dataY)!=len(dataY1):
        print('outlier rejected:')
        print(rejected)
    return [dataX1,dataY1]

In [10]:
def getFinger(dictFin,subject,block,sentence,index):
    #返回点击key的手指标号，1 hand1， 2 hand2， 0 数据不足
    
    if block=='2':
        #如果是block 2，则一定是finger 1
        return 1
    else:
        currentInfo = dictFin[subject][block][sentence]
    
        keyPressingTime = currentInfo['trialtime'][index]
        touchx = currentInfo['touchx'][index]
        touchy = currentInfo['touchy'][index]

        if len(currentInfo['t'])!=0:
            len1 = len(currentInfo['t'][index])
            len2 = len(currentInfo['t'][index+1])
            if len1!=0 and len2!=0:
                #通过点击key之前最后一个figer和点击键之后的第一个finger的连线，和时间位置
                #计算finger在某时间点上的坐标位置
                finTimeBeforeKeyPressing = currentInfo['t'][index][len1-1]
                finTimeAfterKeyPressing = currentInfo['t'][index+1][0]

                #hand 1
                x11,y11 = currentInfo['x1'][index][len1-1],currentInfo['y1'][index][len1-1]
                x12,y12 = currentInfo['x1'][index+1][0],currentInfo['y1'][index+1][0]

                #hand 2
                x21,y21 = currentInfo['x2'][index][len1-1],currentInfo['y2'][index][len1-1]
                x22,y22 = currentInfo['x2'][index+1][0],currentInfo['y2'][index+1][0]

                if finTimeBeforeKeyPressing<=keyPressingTime<=finTimeAfterKeyPressing:
                    #如果当前点击key的时间介于(点击当前key之前最后一个fixation数据行的采样时间)
                    #与(点击当前key之后第一个fixation数据行的采样时间)之间
                    #则通过这两个时间点fixation的位置和时间计算（假设眼球匀速运动）到key pressing的时间点时，fixation的坐标
                    a = (keyPressingTime-finTimeBeforeKeyPressing)/(finTimeAfterKeyPressing-finTimeBeforeKeyPressing)
                    b = (finTimeAfterKeyPressing-keyPressingTime)/(finTimeAfterKeyPressing-finTimeBeforeKeyPressing)
                    #hand 1 点击key时刻的估计位置
                    x1 = b*x11+a*x12
                    y1 = b*y11+a*y12
                    #hand 2 点击key时刻的估计位置
                    x2 = b*x21+a*x22
                    y2 = b*y21+a*y22

                    #判断点击key的时刻，哪个手距离key更近
                    #即为点击key的手指
                    finger1touch = np.sqrt((touchx-x1)*(touchx-x1)+(touchy-y1)*(touchy-y1))
                    finger2touch = np.sqrt((touchx-x2)*(touchx-x2)+(touchy-y2)*(touchy-y2))
                    if finger1touch>finger2touch:
                        return 2
                    else:
                        return 1 

                else:
                    print('----------------------------------------')
                    print('before')
                    print(currentInfo['t'][index])
                    print('after')
                    print(currentInfo['t'][index+1])
                    print('keyPressingTime')
                    print(keyPressingTime)
                    print('----------------------------------------')
                    return 0
            else:
                return 0
        else:
            return 0
    
    

In [11]:
def getLeftHand(subject):
    finger1X = []
    finger2X = []
    block = '1'
    for sentence in dictFin[subject][block].keys():
        for i in range(len(dictFin[subject][block][sentence]['key'])):
            for item1 in dictFin[subject][block][sentence]['x1'][i]:
                finger1X.append(item1)
            for item2 in dictFin[subject][block][sentence]['x2'][i]:
                finger2X.append(item2)
    meanX1 = np.mean(finger1X)
    meanX2 = np.mean(finger2X)
    if meanX1>meanX2:
        return 1
    else:
        return 2

In [12]:
#计算finger efficiency：key之间距离/path长度


block = '1'
probKeyFinger = {}
countUserKeyPressing = {}
for subject in dictFin.keys():
    keys_ = ['q','w','e','r','t','y','u','i','o','p','å','a','s','d','f','g','h','j','k','l','ö','ä','z','x','c','v','b','n','m','<',' ']
    
    countKeyFinger = {}
    probKeyFinger[subject] = {}
    countUserKeyPressing[subject] = {}
    
    for k in keys_:
        countKeyFinger[k] = {}
        probKeyFinger[subject] [k]= {}
        
        countKeyFinger[k]['left'] = 0
        countKeyFinger[k]['right'] = 0
        
        probKeyFinger[subject][k]['left'] = 0
        probKeyFinger[subject][k]['right'] = 0
        
        #每个participant点击每个键几次
        countUserKeyPressing[subject][k] = 0
        
   
    #返回左手是1还是2
    lefthand = getLeftHand(subject)
    #存放左右手指点击每个键的次数
        
    for sentence in dictFin[subject][block].keys():
        for index in range(len(dictFin[subject][block][sentence]['key'])):
            if index!=0 and index!=len(dictFin[subject][block][sentence]['t'])-1:
                currentKey = dictFin[subject][block][sentence]['key'][index]
                activeFinger = getFinger(dictFin,subject,block,sentence,index)
                
                countUserKeyPressing[subject][currentKey] = countUserKeyPressing[subject][currentKey] +1
                
                if activeFinger==lefthand:
                    countKeyFinger[currentKey]['left'] = countKeyFinger[currentKey]['left'] + 1
                else:
                    countKeyFinger[currentKey]['right'] = countKeyFinger[currentKey]['right'] + 1
                    
    for k in keys_:
        if (countKeyFinger[k]['left']+countKeyFinger[k]['right'])==0:
            print(k)
            continue
        else:
            probKeyFinger[subject][k]['left'] = countKeyFinger[k]['left']/(countKeyFinger[k]['left']+countKeyFinger[k]['right'])
            probKeyFinger[subject][k]['right'] = countKeyFinger[k]['right']/(countKeyFinger[k]['left']+countKeyFinger[k]['right'])
        
        
    print(probKeyFinger[subject])
print('----------------------')
print(probKeyFinger)


q
w
f
g
z
x
c
{'q': {'left': 0, 'right': 0}, 'w': {'left': 0, 'right': 0}, 'e': {'left': 0.0, 'right': 1.0}, 'r': {'left': 0.0, 'right': 1.0}, 't': {'left': 0.2972972972972973, 'right': 0.7027027027027027}, 'y': {'left': 0.8333333333333334, 'right': 0.16666666666666666}, 'u': {'left': 1.0, 'right': 0.0}, 'i': {'left': 1.0, 'right': 0.0}, 'o': {'left': 1.0, 'right': 0.0}, 'p': {'left': 1.0, 'right': 0.0}, 'å': {'left': 1.0, 'right': 0.0}, 'a': {'left': 0.0, 'right': 1.0}, 's': {'left': 0.0, 'right': 1.0}, 'd': {'left': 0.0, 'right': 1.0}, 'f': {'left': 0, 'right': 0}, 'g': {'left': 0, 'right': 0}, 'h': {'left': 1.0, 'right': 0.0}, 'j': {'left': 1.0, 'right': 0.0}, 'k': {'left': 1.0, 'right': 0.0}, 'l': {'left': 1.0, 'right': 0.0}, 'ö': {'left': 1.0, 'right': 0.0}, 'ä': {'left': 1.0, 'right': 0.0}, 'z': {'left': 0, 'right': 0}, 'x': {'left': 0, 'right': 0}, 'c': {'left': 0, 'right': 0}, 'v': {'left': 0.9166666666666666, 'right': 0.08333333333333333}, 'b': {'left': 1.0, 'right': 0.0}, 'n'

In [13]:
import math

userEntropyList = {}

for subject in dictFin.keys():
    userEntropyList[subject] = {}
    
    #这个participant的总点击次数
    countAllKeyPressing=0
    for k1 in countUserKeyPressing[subject].keys():
        countAllKeyPressing = countAllKeyPressing + countUserKeyPressing[subject][k1]
    
    for k in keys_:
        if probKeyFinger[subject][k]['left']==0 or probKeyFinger[subject][k]['right']==0:
            continue
        else:
            left = -probKeyFinger[subject][k]['left']*math.log(probKeyFinger[subject][k]['left'],2)
            right = -probKeyFinger[subject][k]['right']*math.log(probKeyFinger[subject][k]['right'],2)
            userEntropyList[subject][k] = {}
            userEntropyList[subject][k]['entropy'] = left+right
            
            #这个participant的CurrentKey点击次数
            countCurrentKeyPressing = countUserKeyPressing[subject][k]
            
            #这个participant的CurrentKey点击频率
            userEntropyList[subject][k]['frequency'] = countCurrentKeyPressing/countAllKeyPressing

print(userEntropyList)
            

    

{'101': {'t': {'entropy': 0.8779620013943912, 'frequency': 0.07676348547717843}, 'y': {'entropy': 0.6500224216483541, 'frequency': 0.024896265560165973}, 'v': {'entropy': 0.41381685030363374, 'frequency': 0.024896265560165973}}, '104': {'e': {'entropy': 0.2580186686648155, 'frequency': 0.05974025974025974}, 'r': {'entropy': 1.0, 'frequency': 0.015584415584415584}, 't': {'entropy': 0.5617526078313282, 'frequency': 0.0987012987012987}}, '105': {'r': {'entropy': 0.8112781244591328, 'frequency': 0.010050251256281407}, 't': {'entropy': 0.9268190639645772, 'frequency': 0.09547738693467336}, 'u': {'entropy': 0.5032583347756457, 'frequency': 0.04522613065326633}, 'i': {'entropy': 0.345117314944953, 'frequency': 0.07788944723618091}, 'a': {'entropy': 0.19143325481419346, 'frequency': 0.08542713567839195}, 's': {'entropy': 0.22228483068568797, 'frequency': 0.07035175879396985}, 'h': {'entropy': 0.9182958340544896, 'frequency': 0.01507537688442211}, 'k': {'entropy': 0.39124356362925566, 'frequenc

In [14]:
#The average entropy of a finger-to-key mapping is then computed 
#as sum over the entropies of each key weighted by the frequency of the corresponding letter. 

In [15]:
userAvgEntropy = {}
for uid in userEntropyList.keys():
    entro = 0
    for key in userEntropyList[uid].keys():
        entro = entro + userEntropyList[uid][key]['entropy']*userEntropyList[uid][key]['frequency']
    userAvgEntropy[uid] = entro

print(userAvgEntropy)
    

{'101': 0.09388104837140317, '104': 0.08644422981008111, '105': 0.42048795886069934, '106': 0.0884247813843311, '107': 0.09938949718126859, '108': 0.42955193929863067, '109': 0.11400136418066398, '110': 0.07159482862787381, '111': 0.37703020402020215, '112': 0.4333661032000567, '113': 0.8477623736482218, '114': 0.16569842346905406, '115': 0.14508176383352211, '116': 0.20389987044909935, '117': 0.08216878428683182, '118': 0.9348212963010704, '120': 0.14687520514691, '121': 0.13562228329981185, '122': 0.3589432975860881, '123': 0.49966130482929144, '124': 0.13163021243532746, '125': 0.23166463540820387, '126': 0.09894783916999074, '127': 0.20887761166270788, '128': 0.2104299522888162, '129': 0.18321448487328662, '130': 0.2882418915477877, '131': 0.46375519144411587, '132': 0.09227883279382529, '133': 0.391177491398259}


In [26]:
block='1'
userIKIList = {}
for subject in dictFin.keys():
    userIKIList[subject] = []
    for sentence in dictFin[subject][block].keys():
        for i in range(len(dictFin[subject][block][sentence]['key'])):
            if i==0:
                continue
            else:
                userIKIList[subject].append(dictFin[subject][block][sentence]['IKI'][i])
ikiList = []
entropyList = []
for k in userIKIList.keys():
    ikiList.append(np.mean(userIKIList[k]))
    entropyList.append(userAvgEntropy[k])

#画散点图
drawGraph(entropyList, ikiList, 'IKI with Key-Finger Entropy', 'Entropy','IKI per participant')

In [25]:
#画散点图 Scatter
def drawGraph(xlist, ylist, title, xlable,ylable):
    import matplotlib.pyplot as plt
    from scipy import stats
  
    t = title
    
    #去掉 IKI 结果中的 outlier
    #temp = rejectOutliers(xlist, ylist)
    #xlist1 = temp[0]
    #ylist1 = temp[1]
    
    
    yfit = []

    slope, intercept, r_value, p_value, std_err = stats.linregress(xlist, ylist)
    for item in xlist:
        yfit.append(slope*item+intercept)

    plt.xlabel(xlable)  
    plt.ylabel(ylable)
    plt.title(t)
    
    plt.plot(xlist, ylist ,'bo', alpha=.5,label='Corr:'+str(round(r_value,2)))
    plt.plot(xlist,yfit,'b', linewidth = 2)
    
    plt.legend()


    plt.savefig(t+".png")